In [1]:
import itertools
import operator
import re
from abc import *
from copy import deepcopy
from operator import itemgetter
from typing import *
import numpy as np
import scipy as sp
import networkx as nx
import mne
import time
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors._dist_metrics import DistanceMetric
from sklearn.utils import shuffle
from typing import List, Tuple
from sklearn.model_selection import train_test_split
from nodestimation.learning.estimation import collect_statistic, \
    compute_importance, collect_cross_statistic, make_selection_map, \
    select, separate_datasets, selected_statistic, choose_best, selected_data, make_feature_selection
from nodestimation.learning.informativeness import CrossInformativeness, Informativeness, SubjectsInformativeness, \
    NodesInformativeness
from nodestimation.learning.networking import sparse_graph, graph_to_hemispheres, hemispheres_division_modularity, \
    metric_for_hemispheres
from nodestimation.processing.features import prepare_features
from nodestimation.project import find_subject_dir, conditions_unique_code
from nodestimation.pipeline import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import nibabel
import nilearn.plotting as nplt
from nodestimation.project.actions import read
import nodestimation as nd
from nodestimation.learning.modification import append_series, promote
import nodestimation.learning.modification as lmd
from nodestimation.project.subject import Subject
from sklearn.preprocessing import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import matplotlib as mpl
from nodestimation.learning.selection import SubjectsStatistic, Wilcoxon, Mannwhitneyu, Test
from scipy.stats import wilcoxon, spearmanr, pearsonr
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from scipy.fftpack import fftfreq, irfft, rfft
from scipy.fftpack import fftfreq, irfft, rfft
from sklearn.linear_model import LinearRegression

ENGEL1 = [
    'B1C2',
    'B1R1',
    'G1R1',
    'G1V2',
    'J1T2',
    'K1V1',
    'L1P1',
    'M1G2',
    'M1N2',
    'O1O2',
    'R1D2',
    'S1A2',
    'S1B1',
    'S1H1',
    'S1U3'
]
ENGEL2 = [
    'L2M1',
    'M2S2',
    'N2K2',
    'P1H2'
]
ENGEL3 = [
    'N3S2',
    'S3R1'
]
ENGEL4 = [
    'K4L2'
]
REJECTED = [
    'S1U3',
    'P1H2'
]

AGE = {
    'B1C2': 28.0,
    'B1R1': 16.0,
    'G1R1': 23.0,
    'G1V2': 5.0,
    'J1T2': 8.0,
    'K1V1': 10.0,
    'K4L2': 14.0,
    'L1P1': 16.0,
    'L2M1': 20.0,
    'M1G2': 8.0,
    'M1N2': 7.0,
    'M2S2': 18.0,
    'N2K2': 30.0,
    'N3S2': 10.0,
    'O1O2': 18.0,
    'R1D2': 6.5,
    'P1H2': 7.0,
    'S1A2': 12.0,
    'S1B1': 17.0,
    'S1H1': 28.0,
    'S3R1': 19.0,
    'S1U3': 15.0,
}

SEX = {
    'B1C2': 'f',
    'B1R1': 'm',
    'G1R1': 'f',
    'G1V2': 'm',
    'J1T2': 'f',
    'K1V1': 'f',
    'K4L2': 'f',
    'L1P1': 'f',
    'L2M1': 'f',
    'M1G2': 'm',
    'M1N2': 'm',
    'M2S2': 'm',
    'N2K2': 'm',
    'N3S2': 'm',
    'O1O2': 'f',
    'R1D2': 'f',
    'P1H2': 'm',
    'S1A2': 'm',
    'S1B1': 'm',
    'S1H1': 'm',
    'S3R1': 'm',
    'S1U3': 'f',
}

ENGEL = dict()
for group, i in zip([ENGEL1, ENGEL2, ENGEL3, ENGEL4], range(4)):
    ENGEL.update({
        subject_name: i+1
        for subject_name in group
    })


def depth(coords: np.ndarray, center: np.ndarray = np.array([0, 0, 0])) -> float:
    return np.sqrt(
        (coords[0] - center[0]) ** 2 +
        (coords[1] - center[1]) ** 2 +
        (coords[2] - center[2]) ** 2
    )

/home/user/.local/lib/python3.9/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
subjects = pipeline(
    methods=['wpli', 'envelope', 'coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'pli2_unbiased', 'wpli2_debiased'],
    freq_bands=(4, 8),
    centrality_metrics=['eigen', 'between', 'degree', 'info']
    )

All computation has been already done, loading of the existing file with the solution...


In [3]:
DATASET = metric_for_hemispheres(subjects, nx.algorithms.global_efficiency)
print('graph done')

graph done


In [ ]:
modularity_env, modularity_wpli = dict(), dict()
freq = '4-8Hz'

for subject in subjects:
    for method in ['wpli', 'envelope']:
        label_names = list(subject.connectomes[freq][method].index)
        mapping = {
            i: label_name
            for i, label_name in zip(
                range(len(label_names)),
                label_names
            )
        }
        G = sparse_graph(
            nx.convert_matrix.from_numpy_matrix(
                subject.connectomes[freq][method].to_numpy()
            )
        )
        G = nx.relabel_nodes(G, mapping)

        modularity = {
            'wpli': modularity_wpli,
            'envelope': modularity_env
        }[method]
        modularity.update({
            subject.name: hemispheres_division_modularity(G)
        })

print(lmd.dict_to_str(modularity_env))
print(lmd.dict_to_str(modularity_wpli))

In [180]:
dataset = DATASET.copy()

for sample in dataset.index:
    subject = sample[:4]
    if subject in REJECTED:
        dataset = dataset.drop(index=sample)

In [160]:
cross_hemispheres_informativeness = CrossInformativeness()
cross_subjects_informativeness = CrossInformativeness()

for _ in range(100):
    hemispheres_informatoveness = Informativeness()
    subjects_informativeness = SubjectsInformativeness()
    acc, spec, sens, pospred, negpred = list(), list(), list(), list(), list()

    for i in range(1000):

        y = dataset['resected'].to_numpy()
        x = dataset[[f'global_efficiency_for_wpli_4-8Hz', f'global_efficiency_for_envelope_4-8Hz']].to_numpy()

        scaler = StandardScaler()
        x = scaler.fit_transform(x)

        samples = [[sample] for sample in dataset.index.tolist()]

        x = np.append(x, samples, axis=1)
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        train_samples, test_samples = x_train[:, 2], x_test[:, 2]
        x_train, x_test = x_train[:, 0:2], x_test[:, 0:2]

        clf = svm.SVC()
        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)

        for predicted, actual, sample, value in zip(pred, y_test, test_samples, x_test):
            hemispheres_informatoveness.informativeness = sample, actual, 'correct' if predicted == actual else 'wrong'
            subjects_informativeness.informativeness = sample, actual, 'correct' if predicted == actual else 'wrong'

    cross_subjects_informativeness.informativeness = subjects_informativeness
    cross_hemispheres_informativeness.informativeness = hemispheres_informatoveness

In [209]:
subjects_acc = cross_subjects_informativeness.acc()

INCLUDED = [
    'B1R1',
    'G1R1',
    'G1V2',
    'L1P1',
    'M1G2',
    'M1N2',
    'O1O2',
    'R1D2',
]

dataset_acc_rel = dataset.copy()

for subject_name, subject_acc in subjects_acc.items():
    if subject_acc < 0.75:
        dataset_acc_rel = dataset_acc_rel.drop(index=subject_name+'_lh')
        dataset_acc_rel = dataset_acc_rel.drop(index=subject_name+'_rh')

acc, spec, sens, pospred, negpred, missed = list(), list(), list(), list(), list(), 0
for i in range(1000):

    y = dataset['resected'].to_numpy()
    x = dataset[[f'global_efficiency_for_wpli_4-8Hz', f'global_efficiency_for_envelope_4-8Hz']].to_numpy()

    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(x, y)

    clf = svm.SVC(kernel='sigmoid')
    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)

    acc.append(accuracy_score(y_test, pred))
    tn, fp, fn, tp = confusion_matrix(y_test, pred, labels=[0,1]).ravel()
    if not any([tn + fp == 0, tp + fn == 0, tn + fn == 0, tp + fp == 0]):
        spec.append(tn / (tn + fp))
        sens.append(tp / (tp + fn))
        negpred.append(tn/(tn + fn))
        pospred.append(tp/(tp + fp))
    else:
        missed += 1

spec = [s for s in spec if not np.isnan(s)]
sens = [s for s in sens if not np.isnan(s)]

print('Results for all')
print('acc: ', np.array(acc).mean())
print('spec: ', np.array(spec).mean())
print('sens: ', np.array(sens).mean())
print(f'{missed = }')

acc, spec, sens, pospred, negpred, missed = list(), list(), list(), list(), list(), 0
for i in range(1000):

    y = dataset_acc_rel['resected'].to_numpy()
    x = dataset_acc_rel[[f'global_efficiency_for_wpli_4-8Hz', f'global_efficiency_for_envelope_4-8Hz']].to_numpy()

    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(x, y)

    clf = svm.SVC(kernel='sigmoid')
    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)

    acc.append(accuracy_score(y_test, pred))
    tn, fp, fn, tp = confusion_matrix(y_test, pred, labels=[0,1]).ravel()
    if not any([tn + fp == 0, tp + fn == 0, tn + fn == 0, tp + fp == 0]):
        spec.append(tn / (tn + fp))
        sens.append(tp / (tp + fn))
        negpred.append(tn/(tn + fn))
        pospred.append(tp/(tp + fp))
    else:
        missed += 1


spec = [s for s in spec if not np.isnan(s)]
sens = [s for s in sens if not np.isnan(s)]
print('\nResults for all above 75%')
print('acc: ', np.array(acc).mean())
print('spec: ', np.array(spec).mean())
print('sens: ', np.array(sens).mean())
print(f'{missed = }')


Results for all
acc:  0.6574
spec:  0.6515704835583745
sens:  0.7229770313776368

Results for all above 75%
acc:  0.9854
spec:  0.9811191211809132
sens:  0.9992276004119465
